In [1]:
import gym
import torch
import pandas as pd
import numpy as np
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"   

from elegantrl.run import *
from neo_finrl.ccxt.data_fetch_ccxt import ccxt_fetch_data
from neo_finrl.ccxt.preprocess_ccxt import preprocess_btc

from neo_finrl.ccxt.env_btc_ccxt import BitcoinEnv
from elegantrl.agent import *

In [2]:
# url = 'Data/crypto_quant/btc/total_data.csv'

# url = 'Data/crypto_quant/btc/total_hour.csv'
# url = 'Data/crypto_quant/btc/total_block.csv'
# url = 'Data/crypto_quant/eth/market_data_total.csv'
url = 'C:/Users/yosou/Documents/BooksWithCode/Binance/Feature-Selection-for-Machine-Learning-master/Data/bnb_santiment.csv'

In [3]:
df = pd.read_csv(url, parse_dates=True).fillna(value=0)
df = df[['time', 'open', 'high', 'low', 'close', 'volume', 'marketcap',
        'active_addresses_1h', 'active_addresses_24h', 'age_consumed',
        'active_addresses_24h_change_1d', 'age_destroyed_change_1d',
        'age_destroyed_change_30d', 'marketcap_usd', 'mvrv_usd_intraday',
        'network_profit_loss', 'nvt_5min', 'transaction_volume', 
        'whale_transaction_count_100k_usd_to_inf', 'whale_transaction_count_1m_usd_to_inf']]

In [4]:
# '''data_fetch'''
# df = ccxt_fetch_data(start = '20200602 00:00:00', end = '20210609 00:00:00',
#                     pair = 'DOGE/USDT', period = '1m')
# df = ccxt_fetch_data(start = '20190418 11:40:00', end = '20210628 00:00:00',
#                     pair = 'BNB/USDT', period = '5m')

In [5]:
df = df.set_index('time')

In [6]:
df = df.dropna()

In [7]:
# from talib.abstract import *

# # df['r'] = np.log(df['close'] / df['close'].shift(1))
# # df['s'] = df['close'].rolling(30).mean()
# # df['m'] = df['r'].rolling(30).mean()
# # df['v'] = df['r'].rolling(30).std()
# # df["daily_return"] = df.close.pct_change(1)

# # Price Transform
# df['AVGPRICE'] = AVGPRICE(df['open'], df['high'], df['low'], df['close'])
# df['MEDPRICE'] = MEDPRICE(df['high'], df['low'])
# df['TYPPRICE'] = TYPPRICE(df['high'], df['low'], df['close'])
# df['WCLPRICE'] = WCLPRICE(df['high'], df['low'], df['close'])

# # Overlap Studies
# df['upperband'], middleband, df['lowerband'] = BBANDS(df['close'], timeperiod=5, matype=0)
# # df['upperband'], df['middleband'], df['lowerband'] = BBANDS(df['close'], timeperiod=5, matype=0)
# df['EMA'] = EMA(df['close'])    
# df['SMA'] = SMA(df['close'], timeperiod=30)
# # df['MIDPOINT'] = MIDPOINT(df['close'], timeperiod=30)
# # df['MIDPRICE'] = MIDPRICE(df['high'], df['low'], timeperiod=30)

# # Momentum Indicators
# # df['MACD'], df['MACDSIGNAL'], df['MACDHIST'] = MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
# df['RSI'] = RSI(df['close'], timeperiod=30)
# # df['FASTK'], df['FASTD'] = STOCHRSI(df['close'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
# df['CCI'] = CCI(df['high'], df['low'], df['close'], timeperiod=30)
# df['DX'] = DX(df['high'], df['low'], df['close'], timeperiod=30)
# # df['slowk'], df['slowd'] = STOCH(df['high'], df['low'], df['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
# # df['fastkf'], df['fastdf'] = STOCHF(df['high'], df['low'], df['close'], fastk_period=5, fastd_period=3, fastd_matype=0)

#  # Volatility Indicator Functions
# # df['ATR'] = ATR(df, timeperiod=30)
# # df['ATR%'] = df['ATR'] / df['close']
# # df['NATR'] = NATR(df, timeperiod=30)
# # df['TRANGE'] = TRANGE(df)

In [8]:
# def create_lag_data(lag=30):
#     df_lagged = df.copy()
#     for window in range(1, lag + 1):
#         shifted = df.shift(window)
#         shifted.columns = [x + "_lag_" + str(window) for x in df.columns]
#         df_lagged = pd.concat((df_lagged, shifted), axis=1)
#     df_lagged = df_lagged.dropna()
#     return df_lagged

In [9]:
# df = create_lag_data()

In [10]:
# df = df.drop([ 'long_liquidations_all_exchange', 'short_liquidations_all_exchange',
#                'long_liquidations_usd_all_exchange', 'short_liquidations_usd_ftx',
#                'short_liquidations_usd_all_exchange', 'long_liquidations_binance',
#                'short_liquidations_binance', 'long_liquidations_usd_binance',
#                'short_liquidations_usd_binance', 'long_liquidations_bitmex',
#                'short_liquidations_bitmex', 'long_liquidations_usd_bitmex',
#                'short_liquidations_usd_bitmex', 'long_liquidations_deribit',
#                'short_liquidations_deribit', 'long_liquidations_usd_deribit',
#                'short_liquidations_usd_deribit', 'long_liquidations_ftx',
#                'short_liquidations_ftx', 'long_liquidations_usd_ftx'], axis=1)

In [11]:
# df = df[350000:]
df

,open,high,low,close,volume,marketcap,active_addresses_1h,active_addresses_24h,age_consumed,active_addresses_24h_change_1d,age_destroyed_change_1d,age_destroyed_change_30d,marketcap_usd,mvrv_usd_intraday,network_profit_loss,nvt_5min,transaction_volume,whale_transaction_count_100k_usd_to_inf,whale_transaction_count_1m_usd_to_inf
time,,,,,,,,,,,,,,,,,,,
2019-04-18 11:40:00+00:00,20.906860,20.906860,20.906860,20.906860,2.884547e+08,2.951536e+09,2.0,2.0,0.000000,0.000000,0.000000,0.000000,2.951536e+09,1.000942,0.000000,1.000000,100.000000,0.0,0.0
2019-04-18 12:05:00+00:00,20.721984,20.721984,20.721984,20.721984,2.891850e+08,2.925436e+09,4.0,4.0,0.038113,0.000000,0.000000,0.000000,2.925436e+09,0.992190,-0.210379,1.000000,2.000000,0.0,0.0
2019-04-18 12:10:00+00:00,20.716149,20.738513,20.716149,20.738513,2.894371e+08,2.927770e+09,6.0,6.0,0.043437,0.000000,0.000000,0.000000,2.927770e+09,0.992546,-0.210379,1.000000,2.000000,0.0,0.0
2019-04-18 12:30:00+00:00,20.849631,20.849631,20.849631,20.849631,2.917608e+08,2.943457e+09,7.0,7.0,0.036343,0.000000,0.000000,0.000000,2.943457e+09,0.998453,-0.105190,1.000000,1.000000,0.0,0.0
2019-04-18 12:45:00+00:00,20.834050,20.834050,20.834050,20.834050,2.935855e+08,2.941257e+09,7.0,7.0,0.027755,0.000000,0.000000,0.000000,2.941257e+09,0.997707,0.000000,1.000000,0.998750,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-27 23:40:00+00:00,288.660384,288.670903,288.660384,288.670903,1.504580e+09,4.429161e+10,1663.0,1663.0,797.953762,-0.016559,-0.326509,-0.671526,4.429161e+10,2.535069,-363.201367,1473.953442,708.710953,0.0,0.0
2021-06-27 23:45:00+00:00,287.931333,288.224865,287.931333,288.224865,1.502282e+09,4.422318e+10,1617.0,1617.0,1755.131805,-0.017571,-0.325700,-0.671528,4.422318e+10,2.529910,-8070.276541,1468.067410,934.299174,0.0,0.0
2021-06-27 23:50:00+00:00,290.097753,290.492024,290.097753,290.492024,1.512336e+09,4.457103e+10,1580.0,1580.0,3968.247490,-0.017266,-0.321980,-0.671521,4.457103e+10,2.549349,-9757.745641,1446.037828,3481.806445,2.0,0.0


In [12]:
data_ary = preprocess_btc(df, is_lag=False)

                                open       high        low      close  \
time                                                                    
2019-04-18 11:40:00+00:00  20.906860  20.906860  20.906860  20.906860   
2019-04-18 12:05:00+00:00  20.721984  20.721984  20.721984  20.721984   
2019-04-18 12:10:00+00:00  20.716149  20.738513  20.716149  20.738513   
2019-04-18 12:30:00+00:00  20.849631  20.849631  20.849631  20.849631   
2019-04-18 12:45:00+00:00  20.834050  20.834050  20.834050  20.834050   

                                volume     marketcap  active_addresses_1h  \
time                                                                        
2019-04-18 11:40:00+00:00  288454674.0  2.951536e+09                  2.0   
2019-04-18 12:05:00+00:00  289185042.0  2.925436e+09                  4.0   
2019-04-18 12:10:00+00:00  289437126.0  2.927770e+09                  6.0   
2019-04-18 12:30:00+00:00  291760825.0  2.943457e+09                  7.0   
2019-04-18 12:45:00+00:00 

In [13]:
df = df.reset_index()

In [14]:
data_ary.shape

(208393, 19)

In [15]:
gpu_id = 1
args = Arguments(agent=None, env=None, gpu_id=gpu_id)
args.agent = AgentD3QN()

In [16]:
'''choose environment'''
args.env = BitcoinEnv(processed_ary = data_ary, if_train=True, if_test=False)
args.env_eval = BitcoinEnv(processed_ary = data_ary, if_train=False, if_test=False)
args.net_dim = 2 ** 12 # change a default hyper-parameters
args.batch_size = 2 ** 11

In [ ]:
train_and_evaluate(args)

| GPU id: 1, cwd: ./AgentD3QN/BitcoinEnv_1
| Remove history
ID      Step      MaxR |    avgR      stdR       objA      objC |  avgS  stdS


In [ ]:
args = Arguments(agent=None, env=None, gpu_id=gpu_id)
args.agent = AgentD3QN()
args.env = BitcoinEnv(processed_ary = data_ary, if_train=False, if_test=True)
args.net_dim = 2 ** 12 # change a default hyper-parameters
args.batch_size = 2 ** 11
args.if_remove = False
args.cwd = './AgentD3QN/BitcoinEnv_'+str(gpu_id)
args.init_before_training()

In [ ]:
prediction = args.env.trade_prediction(args, torch, df)

In [ ]:
args.env.backtest_plot(prediction, df)

In [ ]:
# Draw the graph
import matplotlib.pyplot as plt
%matplotlib inline

BitcoinEnv(processed_ary = data_ary, if_train=False, if_test=True)\
.draw_cumulative_return(self = args.env, args = args, _torch = torch)